In [1]:
import sys
print("현재 커널 파이썬:", sys.executable)

현재 커널 파이썬: /usr/bin/python3.11


In [2]:
#!{sys.executable} -m pip install transformers bert-score peft accelerate

In [3]:
# !pip install bert-score

In [4]:
# !pip install peft

In [5]:
# !pip install hf_transfer

In [6]:
#!{sys.executable} -m pip install -U hf_transfer

In [7]:
import sys
print(sys.executable)  # 지금 쓰는 파이썬 경로 확인

!{sys.executable} -m pip show hf_transfer

/usr/bin/python3.11


Name: hf_transfer
Version: 0.1.9
Summary: Speed up file transfers with the Hugging Face Hub.
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [8]:
import hf_transfer
print(hf_transfer.__version__)

0.1.9


In [9]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from bert_score import score
from peft import AutoPeftModelForCausalLM
import torch

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
adapter_model_name = "poketmon/ax-trained-model-v3-2"

base_model_name = "skt/A.X-4.0-Light"

In [11]:
base_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype="auto", device_map="auto", trust_remote_code=True)
base_pipe = pipeline("text-generation", model=base_model, tokenizer=base_tokenizer, return_full_text=False)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:15<00:00,  5.12s/it]
Device set to use cuda:0


In [12]:
print(f"🚀 베이스 모델 로드 중... (base={base_model_name})")
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True   
)

print(f"🚀 LoRA 어댑터 로드 중... (adapter={adapter_model_name})")
model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_model_name,
    dtype=torch.float16,     
    trust_remote_code=True
)

print("🚀 pipeline 생성 중...")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    return_full_text=False, 
)

🚀 베이스 모델 로드 중... (base=skt/A.X-4.0-Light)


🚀 LoRA 어댑터 로드 중... (adapter=poketmon/ax-trained-model-v3-2)


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.08s/it]
Device set to use cuda:0


🚀 pipeline 생성 중...


In [13]:
passage = """
지현: 어… 다 들어왔죠? 오늘은 좀 깁니다. 데이터 레이크 전면 재설계랑 ETL 리워크, 그리고 인프라 케파 다시 짜는 건데요. 요즘 장애가 너무 잦아서 그냥 핫픽스 때려넣고 버티는 수준이 아니라 구조 자체를 갈아엎어야 하는 상황이에요.
하람: 네. 어제 밤에도 레이크 읽기 딜레이 터졌어요. hudi 파일 조각 천 개 넘어서 완전 데이터 블로트 나고, 플링크는 백프레셔 걸려서 ETL이 세 시간씩 밀리고… 그냥 전체가 꽉 막혀 있어요.
민수: AI 쪽도 요즘 콜드 스타트 너무 심해요. 인덱싱이 실시간이 아니고 그냥 어제 데이터 수준이라 RAG라고 부르기도 민망해요. 임베딩은 빨리 되는데 레이크에서 파일 긁어올 때 완전 막혀요.
윤기: 그거 컨테이너 노드에서 I/O 터진 거여서 그래요. 지난번에도 그랬잖아요. executor가 죽었는데, 헬스체크가 제대로 안 돼서 컨테이너 오펀 만드는 바람에 자원만 잡아먹고 있는 상태였어요.
다은: 맞아요. 오펀 생기면 진짜 답 없어요. 스케일아웃도 먹히지도 않고, 워커 노드도 죽은 척 살아있는 애들이 있으니까 autoscaler도 헷갈리고… 결국 ETL 전체가 밀리죠.
지현: 그러면 ETL은 병렬도가 낮아서 밀리는 게 1번이고, 컨테이너 오펀이 케파를 갉아먹는 게 2번이라는 거죠?
하람: 네. 그리고 파티션도 문제예요. 카프카 파티션 400개 넘는데 플링크 parallelism이 32예요. 이건 그냥 스프린트 때 스코프 크리프 난 결과죠.
민수: AI는 또 AI대로 스키마 터진 거 때문에 힘들어요. RAG 인덱싱이 엔드투엔드로 가야 되는데, 레이크에 들어오는 JSON 필드가 매번 달라요. 원래 key-value 몇 개만 쓰던 걸 갑자기 고객사별로 구조가 다르게 들어오니까 모델이 헷갈릴 수밖에 없어요.
소연: QA에서도 계속 문제 생겨요. 같은 쿼리인데 2초 나왔다가 10초 나왔다가… 완전 슬로우쿼리 뽑기운이에요. 안정성이 아예 없어요.
해준: 인프라 입장에서 보면 지금 케파는 이미 끝났어요. CPU는 100 찍고, 디스크 I/O는 울면서 버티고, 레이크 스캔은 하루 종일 풀스캔 찍고 있고… 지금 상태로는 다음 달 캠페인 트래픽 못 받습니다.
지현: 그러면 케파 재설계를 아예 오늘 확정해야겠네요. 더 미룰 수가 없어요.
하람: 우선순위는 레이크 구조 정리, ETL 병렬도 확장, ingestion 스케일 아웃, 인프라 워크로드 분리 순서가 맞아요.
다은: 레이크 쪽은 GC 주기를 늘려야 해요. 지금 delete marker 5일은 너무 짧아서 historical query가 깨져요. 최소 14일로 올려야 해요.
지현: 14일이면 비용 올라가죠?
다은: 네. 근데 지금 성능이 죽어 있는데 비용 아껴봤자 의미 없어요.
민수: RAG 쪽도 이참에 완전 리팩해야 해요. 인덱싱 비용이 지난달 대비 두 배예요. 다 조각 파일 때문이죠.
윤기: 인프라도 워크로드 분리해야죠. 지금은 ETL, 레이크, RAG, 임베딩이 다 같은 클러스터에서 놀고 있으니까 하나 죽으면 다 죽어요.
해준: 케파 잡을 때 레이크는 고정 케파로 가고, ETL은 좀 공격적으로 auto-scaling 걸고, RAG랑 인퍼런스는 트래픽 기준으로 가야 돼요. 지금처럼 한 스택에 다 넣는 방식은 이제 한계예요.
소연: QA 기준도 바꿔야 해요. 지금은 전체 레이턴시만 보는데, ingestion 레이턴시, 레이크 읽기 레이턴시, 인덱싱 레이턴시, RAG 응답 레이턴시 이렇게 쪼개서 봐야 어디서 병목인지 잡혀요.
지현: 좋습니다. 그럼 오늘 목표는 여기까지. 제가 오늘 저녁까지 정리해서 데크 만들고, 내일 스탠드업에서 공유할게요.
민수: 전 인텐트 스키마부터 먼저 잡아볼게요. 요즘 자꾸 꼬여요.
하람: 저희는 compaction tuning 바로 들어갈게요.
다은: 카프카 파티션도 이번 주 안에 손대볼게요.
해준: 인프라는 워크로드 분리 초안 만들어서 내일까지 드릴게요.
지현: 오케이 다들 고생 많았습니다. 회의 종료할게요.

"""

In [14]:
from datetime import date

current_date = date.today().isoformat()

In [15]:
record_id = "pan_text"

In [16]:
system_prompt_template = """
[시스템]
현재 날짜는 {current_date} 입니다. 이 날짜를 기준으로 'due_date'를 계산하십시오.
'오늘' = {current_date}
'내일' = (오늘 + 1일)
'이번 주 금요일' = (오늘 기준 금요일 날짜)

[지시]
[입력 전문]만을 근거로 "summary", "agendas", "tasks" 필드를
요청된 **JSON 형식**으로만 출력하라.
**다른 설명이나 텍스트, 마크다운(` ```json `)을 포함하지 말고, 순수한 JSON 객체 하나만 생성해야 한다.**

[입력 전문]
{transcript}

[출력 형식 (JSON)]
아래 JSON 구조를 정확히 따르되, 실제 값은 전문 기반으로 채워라.

{{
  "id": "{record_id}",
    "summary": {{
      "agenda_1": {{
        "who": "...",
        "what": "...",
        "when": "...",
        "where": "...",
        "why": "...",
        "how": "...",
        "how_much": "...",
        "how_long": "..."
      }},
      "agenda_2": {{
        "who": "...",
        "what": "...",
        "when": "...",
        "where": "...",
        "why": "...",
        "how": "...",
        "how_much": "...",
        "how_long": "..."
      }}
    }}
  }},
  "agendas": [
    {{
      "title": "안건 제목",
      "description": "안건 설명 (없으면 null)"
    }}
  ],
  "tasks": [
    {{
      "description": "태스크 내용",
      "assignee": "담당자 이름",
      "due": "마감일 표현",
      "due_date": "YYYY-MM-DD"
    }}
  ]
}}

[규칙]
1. 반드시 유효한 JSON 객체 하나만 출력한다.
2. id에는 시스템에서 전달된 {record_id}를 그대로 사용한다.
3. summary.agendas와 agendas와 tasks는 형식 개수 상관 없이 추출할 수 있는대로 뽑는다.
4. summary.agendas:
   - summary.agenda_1, summary.agenda_2 … 의 모든 필드는 '전문에 나오는 단어만' 사용해야 한다.
   - 각 agenda도 5W3H 8개 항목을 포함한다.
   - 전문에 직접 언급되지 않은 표현을 절대 생성하지 말고, 해당 항목은 null 로 설정한다.
5. agendas:
   - 안건 리스트를 title/description 형태로 제공한다.
   - 안건 리스트 없으면 [].
   - description은 없으면 null.
   - summary의 agenda_1, agenda_2… 와 순서를 동일하게 맞춘다.
6. tasks:
   - 태스크 리스트 없으면 [].
   - due: 전문에 나온 마감일 표현 (예: "오늘 오후", "다음 주 초"). 없으면 *(언급 없음)*.
   - due_date: [시스템]의 {current_date} 기준 계산. 계산 불가 또는 due 없음 → null.
7. 근거 없는 정보 생성 금지.
8. 출력은 JSON 한 덩어리로. 부가 텍스트 금지.

위 규칙을 모두 지켜 JSON 결과만 출력하라.
"""

In [17]:
prompt = system_prompt_template.format(
    current_date=current_date,
    record_id="syn_pan",
    transcript=passage
)

In [18]:
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": passage}
]

output = pipe(messages, max_new_tokens=1024, temperature=0.1)
prediction = output[0]['generated_text'].strip()
print(prediction)

```json
{
  "id": "syn_pan",
  "summary": {
    "agenda_1": {
      "who": "지현",
      "what": "데이터 레이크 전면 재설계",
      "when": "오늘",
      "where": "회의실",
      "why": "장애 빈도 증가",
      "how": "구조적 문제 해결",
      "how_much": "핫픽스 수준",
      "how_long": "하루 종일"
    },
    "agenda_2": {
      "who": "하람",
      "what": "ETL 리워크",
      "when": "오늘",
      "where": "회의실",
      "why": "병렬도 부족",
      "how": "병렬도 확장",
      "how_much": "세 시간 지연",
      "how_long": "하루 종일"
    }
  },
  "agendas": [
    {
      "title": "데이터 레이크 구조 재설계",
      "description": "장애 빈도 증가로 구조적 문제 해결 필요"
    },
    {
      "title": "ETL 병렬도 확장",
      "description": "병렬도 부족으로 인한 지연 문제 해결"
    }
  ],
  "tasks": [
    {
      "description": "인텐트 스키마 정리",
      "assignee": "민수",
      "due": "오늘 오후",
      "due_date": "2025-11-22"
    },
    {
      "description": "컴팩션 튜닝",
      "assignee": "하람",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "카프카 파티션 조정",
      "assignee": "다은",

In [22]:
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": passage}
]

output = base_pipe(messages, max_new_tokens=1024, temperature=0.1)
prediction = output[0]['generated_text'].strip()
print(prediction)

```json
{
  "id": "syn_pan",
  "summary": {
    "agenda_1": {
      "who": "지현",
      "what": "데이터 레이크 전면 재설계 및 ETL 리워크",
      "when": "오늘",
      "where": "회의실",
      "why": "장애 빈도 증가 및 구조 개선 필요",
      "how": "핫픽스 대신 구조 변경",
      "how_much": "전체 구조 재설계",
      "how_long": "오늘"
    },
    "agenda_2": {
      "who": "하람",
      "what": "레이크 읽기 딜레이 및 플링크 백프레셔 문제 해결",
      "when": "어제 밤",
      "where": "시스템",
      "why": "데이터 블로트 및 ETL 지연",
      "how": "Hudi 파일 조각 수 관리 및 플링크 설정 조정",
      "how_much": "Hudi 파일 조각 1000개 이하 유지",
      "how_long": "지속적 모니터링 필요"
    }
  },
  "agendas": [
    {
      "title": "데이터 레이크 재설계 및 ETL 리워크",
      "description": "장애 빈도 증가로 인해 데이터 레이크 구조 전면 재설계 및 ETL 프로세스 리워크 필요"
    },
    {
      "title": "레이크 읽기 딜레이 및 플링크 백프레셔 문제",
      "description": "Hudi 파일 조각 수 증가와 플링크 백프레셔로 인한 ETL 지연 문제 해결"
    }
  ],
  "tasks": [
    {
      "description": "레이크 GC 주기 5일에서 14일로 조정",
      "assignee": "다은",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {


In [20]:
refs = [
"""
{{
  "id": "{record_id}",
  "summary": {{
    "agenda_1": {{
      "who": "데이터팀(하람), 플랫폼팀(다은), AI팀(민수)",
      "what": "Hudi 기반 데이터 레이크 구조 전면 재설계 및 파일 조각 폭증으로 인한 성능 저하 문제",
      "when": "최근 지속적으로 발생하며 어제 밤에도 딜레이 발생",
      "where": "Hudi 기반 데이터 레이크",
      "why": "compaction 밀림, GC 주기 과도 단축, 레거시 노드 IO 저하로 인해 파일 조각이 수천 개 수준으로 증가했기 때문",
      "how": "GC 주기를 14일 이상으로 확대하고 compaction 튜닝 및 레이크 구조 분리를 통해 읽기 경로를 재설계",
      "how_much": "플링크·ETL·RAG 전체가 수 시간씩 밀리고 슬로우쿼리가 급증",
      "how_long": "compaction 개선은 다음 주 중반까지를 목표로 함"
    }},
    "agenda_2": {{
      "who": "데이터 플랫폼팀(다은), 데이터팀(하람)",
      "what": "ETL 병렬도 부족과 컨테이너 오펀으로 인한 파이프라인 지연 문제",
      "when": "어제도 ETL이 약 3시간 지연되는 등 최근 반복적으로 발생",
      "where": "Spark 기반 ETL 파이프라인",
      "why": "파티션·병렬도 불균형과 종료 신호 전달 실패로 driver가 죽고 executor만 남는 오펀 상태가 발생하기 때문",
      "how": "병렬도 확장, 파티션 재설계, driver 안정성 보강 및 워크로드 분리",
      "how_much": "autoscaling 효과가 무력화되고 ETL 전체가 밀리며 다른 잡에도 영향을 줌",
      "how_long": "병렬도 조정 작업을 이번 주 안에 착수할 계획"
    }},
    "agenda_3": {{
      "who": "플랫폼팀(다은), 데이터팀(하람)",
      "what": "Flink 병렬도와 Kafka 파티션 불균형으로 인한 backpressure 문제",
      "when": "최근 수 주간 지속적으로 관측",
      "where": "Kafka → Flink ingest 경로",
      "why": "Kafka 파티션은 400개인데 Flink 병렬도는 32 수준에 머물러 병목이 발생하고 레거시 설정이 유지되고 있기 때문",
      "how": "Flink 병렬도 확장, Kafka 파티션 재정의 및 ingest 구조 리워크",
      "how_much": "ETL ingest가 밀리며 전체 파이프라인이 수 시간 단위로 지연",
      "how_long": "이번 주 내에 파티션 조정 및 병렬도 확장 작업을 시작할 계획"
    }},
    "agenda_4": {{
      "who": "AI팀(민수)",
      "what": "RAG 인덱싱 지연 및 스키마 불안정으로 인한 실시간성 붕괴",
      "when": "어제 기준 약 3시간 인덱싱 지연",
      "where": "데이터 레이크 → 임베딩 → 인덱서 경로",
      "why": "레이크 I/O 병목과 고객사별 JSON 스키마 구조가 제각각이라 인덱싱 및 처리 체인이 불안정하기 때문",
      "how": "인텐트 스키마 정규화, RAG 경로 분리, 레이크 안정화 이후 인덱싱 파이프라인 재정비",
      "how_much": "현재는 실시간 RAG가 어려워 사실상 배치 처리 수준으로 인식되고 있음",
      "how_long": "레이크 및 ETL 안정화 이후 단계적으로 개선될 것으로 예상"
    }},
    "agenda_5": {{
      "who": "데이터 플랫폼팀(다은)",
      "what": "Hudi GC(delete marker) 주기 확장 필요",
      "when": "최근 historical query 오류가 반복적으로 확인됨",
      "where": "Hudi delete marker 정책",
      "why": "delete marker 기간이 5일로 너무 짧아 파일 정렬 타이밍과 충돌하며 log/base sync 오류를 유발하기 때문",
      "how": "GC 주기를 최소 14일 이상으로 연장하여 compaction과 GC가 충돌하지 않도록 정책 조정",
      "how_much": "레이크 read consistency가 떨어지고 검색·RAG 결과 정확도가 저하됨",
      "how_long": "이번 주 내 정책 변경이 가능하다고 논의됨"
    }},
    "agenda_6": {{
      "who": "인프라팀(해준), 데이터·AI팀",
      "what": "인프라 케파 한계와 워크로드 분리 필요성",
      "when": "현재까지 지속적으로 케파가 바닥인 상태",
      "where": "레이크·ETL·RAG·임베딩이 모두 올라간 공용 클러스터",
      "why": "모든 워크로드가 단일 클러스터에 집적되어 있고 노드 warm-up이 느려 auto-scaling 효과가 사라졌기 때문",
      "how": "레이크는 고정 케파, ETL은 aggressive auto-scaling, RAG·AI는 트래픽 기반 별도 클러스터로 분리하는 방향으로 재설계",
      "how_much": "CPU 100% 근접, 디스크 IO 포화로 캠페인 트래픽을 제대로 수용하지 못하는 수준",
      "how_long": "분기 전체를 스코프로 하는 중장기 작업으로 추진"
    }},
    "agenda_7": {{
      "who": "QA팀(소연)",
      "what": "QA 모니터링 기준 세분화 및 관측 지표 재정의 필요",
      "when": "최근 레이턴시 변동이 심해지면서 명확히 드러남",
      "where": "ingestion → 레이크 read → 인덱싱 → RAG 응답 전체 경로",
      "why": "현재는 단일 latency 값만 모니터링하여 어느 구간에서 병목이 발생하는지 알 수 없기 때문",
      "how": "단계별 latency 측정 기준 재정의, 구간별 지표 분리 및 QA 테스트 플로우 재설계",
      "how_much": "QA 검증 신뢰도가 떨어지고 문제 원인 파악에 시간이 많이 소요됨",
      "how_long": "구조 개편 작업과 병행하여 진행할 필요가 있음"
    }}
  }},
  "agendas": [
    {{
      "title": "ETL 병렬도 부족 및 백프레셔 지속 문제",
      "description": "Flink parallelism이 Kafka 파티션 수를 따라가지 못해 ETL ingest 병목과 backpressure가 지속적으로 발생하고 있으며, Spark 기반 ETL에서도 병렬도 부족·파티션 불균형으로 지연이 반복되고 있음."
    }},
    {{
      "title": "데이터 레이크 성능 저하 및 파일 조각 폭증",
      "description": "Hudi compacting 밀림과 GC 주기 과도 단축으로 인해 조각 파일이 폭증하고, 레이크 스캔 비용과 슬로우쿼리가 급증해 전체 파이프라인 성능이 저하되고 있음."
    }},
    {{
      "title": "컨테이너 오펀 및 인프라 케파 소진",
      "description": "잘못된 종료·헬스체크로 driver가 죽고 executor만 남는 오펀 컨테이너가 발생해 리소스를 잠식하고, 공용 클러스터의 CPU·디스크 IO가 포화 상태에 도달하여 auto-scaling 효과가 거의 없는 상태."
    }},
    {{
      "title": "RAG 실시간성 붕괴",
      "description": "RAG 인덱싱 및 레이크 스캔 지연으로 실시간 검색이 불가능해지고, 고객사별 스키마 불일치까지 겹쳐 RAG 응답성이 크게 떨어진 상황."
    }},
    {{
      "title": "워크로드 분리 및 관측 지표 재정의 필요",
      "description": "레이크·ETL·RAG·임베딩이 단일 클러스터에 섞여 있어 한 워크로드의 병목이 전체 서비스로 전파되며, 지표도 단일 latency만 보는 구조라 ingestion, 레이크 read, 인덱싱, RAG 응답을 단계별로 분리해 모니터링해야 함."
    }}
  ],
  "tasks": [
    {{
      "description": "Hudi compacting 튜닝 작업 착수",
      "assignee": "하람",
      "due": "이번 주",
      "due_date": "2025-11-21"
    }},
    {{
      "description": "Kafka 파티션 튜닝 및 재설계 시작",
      "assignee": "다은",
      "due": "이번 주",
      "due_date": "2025-11-21"
    }},
    {{
      "description": "인프라 워크로드 분리 구조 초안 작성",
      "assignee": "해준",
      "due": "내일까지",
      "due_date": "2025-11-21"
    }},
    {{
      "description": "RAG용 인텐트 스키마 정리 및 정규화",
      "assignee": "민수",
      "due": "기한 미정",
      "due_date": null
    }},
    {{
      "description": "오늘 회의 내용을 데크로 정리 및 공유",
      "assignee": "지현",
      "due": "오늘 저녁",
      "due_date": "2025-11-20"
    }}
  ]
}}

   
"""]
cands = [prediction]

In [21]:
print("🚀 BERTScore 계산 중...")
P, R, F1 = score(cands, refs, lang="ko", verbose=True)
print("\n=== 최종 결과 ===")
print(f"Precision (정밀도): {P.mean().item():.4f}")
print(f"Recall    (재현율): {R.mean().item():.4f}")
print(f"F1 Score  (조화평균): {F1.mean().item():.4f}")

🚀 BERTScore 계산 중...


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 13.60it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 43.91it/s]

done in 0.11 seconds, 9.45 sentences/sec

=== 최종 결과 ===
Precision (정밀도): 0.7929
Recall    (재현율): 0.7570
F1 Score  (조화평균): 0.7745
